In [78]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.model_selection import train_test_split

from catboost import Pool, CatBoostClassifier, cv, CatboostIpythonWidget

from auto_ml import Predictor
from auto_ml.utils_models import load_ml_model

In [5]:
test = pd.read_csv("./data/criminal_test.csv")
train= pd.read_csv("./data/criminal_train.csv")

In [10]:
featuers = train.columns[train.columns != "Criminal"]
X = train[featuers]
Y = train['Criminal']
list = ['PERID']

In [11]:
x = train.drop(list,axis=1)
y = test.drop(list,axis=1)

In [12]:
list1 = ['TROUBUND']
x = x.drop(list1,axis=1)
y = y.drop(list1,axis=1)
x = x.drop('Criminal',axis=1)

In [19]:
X_train, X_validation, y_train, y_validation = train_test_split(x, Y, train_size=0.85, random_state=1234)

C:\Anaconda3\envs\ml\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
model = XGBClassifier()
model.fit(x,Y)

In [136]:
column_descriptions = {
  'Criminal': 'output'
}
training_params = {'n_estimators': 2000, 'learning_rate': 0.2, 'num_leaves': 14
                   , 'lambda_l2': 0.001, 'histogram_pool_size': 16384}
training_params2 = {'n_estimators': 2000, 'learning_rate': 0.2, 'num_leaves': 10, 'lambda_l2': 0.001, 'histogram_pool_size': 16384}
ml_predictor = Predictor(type_of_estimator='classifier'
                         , column_descriptions=column_descriptions)

ml_predictor.train(train, model_names=['LGBMClassifier'], training_params=training_params)

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at http://auto.ml
Now using the model training_params that you passed in:
{'n_estimators': 2000, 'learning_rate': 0.2, 'num_leaves': 14, 'lambda_l2': 0.002, 'histogram_pool_size': 16384}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'n_estimators': 2000, 'learning_rate': 0.2, 'num_leaves': 14, 'lambda_l2': 0.002, 'histogram_pool_size': 16384}
Running basic data cleaning
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{'n_estimators': 2000, 'learning_rate': 0.2, 'num_leaves': 14, 'lambda_l2': 0.002, 'histogram_pool_size': 16384}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'n_estimators':

C:\Anaconda3\envs\ml\lib\site-packages\lightgbm\basic.py:1029: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is []
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	random_holdout_set_from_training_data's binary_logloss: 0.536578
Training until validation scores don't improve for 100 rounds.
[2]	random_holdout_set_from_training_data's binary_logloss: 0.431746
[3]	random_holdout_set_from_training_data's binary_logloss: 0.356623
[4]	random_holdout_set_from_training_data's binary_logloss: 0.300984
[5]	random_holdout_set_from_training_data's binary_logloss: 0.258593
[6]	random_holdout_set_from_training_data's binary_logloss: 0.226031
[7]	random_holdout_set_from_training_data's binary_logloss: 0.200565
[8]	random_holdout_set_from_training_data's binary_logloss: 0.180488
[9]	random_holdout_set_from_training_data's binary_logloss: 0.164753
[10]	random_holdout_set_from_training_data's binary_logloss: 0.152097
[11]	random_holdout_set_from_training_data's binary_logloss: 0.141988
[12]	random_holdout_set_from_training_data's binary_logloss: 0.133673
[13]	random_holdout_set_from_training_data's binary_logloss: 0.127031
[14]	random_holdout_set_from_trainin

[134]	random_holdout_set_from_training_data's binary_logloss: 0.100361
[135]	random_holdout_set_from_training_data's binary_logloss: 0.100425
[136]	random_holdout_set_from_training_data's binary_logloss: 0.100482
[137]	random_holdout_set_from_training_data's binary_logloss: 0.10051
[138]	random_holdout_set_from_training_data's binary_logloss: 0.100563
[139]	random_holdout_set_from_training_data's binary_logloss: 0.100562
[140]	random_holdout_set_from_training_data's binary_logloss: 0.100575
[141]	random_holdout_set_from_training_data's binary_logloss: 0.10066
[142]	random_holdout_set_from_training_data's binary_logloss: 0.100717
[143]	random_holdout_set_from_training_data's binary_logloss: 0.100659
[144]	random_holdout_set_from_training_data's binary_logloss: 0.100546
[145]	random_holdout_set_from_training_data's binary_logloss: 0.100557
[146]	random_holdout_set_from_training_data's binary_logloss: 0.100855
Early stopping, best iteration is:
[46]	random_holdout_set_from_training_data's

C:\Anaconda3\envs\ml\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


The printed list will only contain at most the top 100 features.
+---------+----------------+--------------+---------------+-------------------+-------------------+-----------+-----------+-----------+-----------+
|         | Feature Name   |   Importance |         Delta |   FR_Decrementing |   FR_Incrementing |   FRD_abs |   FRI_abs |   FRD_MAD |   FRI_MAD |
|---------+----------------+--------------+---------------+-------------------+-------------------+-----------+-----------+-----------+-----------|
| 35.0000 | IIOTHHLT       |            0 |        1.3674 |            0.0000 |            0.0000 |    0.0000 |    0.0000 |    0.0000 |    0.0000 |
| 21.0000 | HLNVCOST       |            0 |        6.7521 |            0.0000 |            0.0000 |    0.0000 |    0.0000 |    0.0000 |    0.0000 |
| 22.0000 | HLNVOFFR       |            0 |        6.6605 |            0.0000 |            0.0000 |    0.0000 |    0.0000 |    0.0000 |    0.0000 |
| 23.0000 | HLNVREF        |            0 |    

In [ ]:
# test_score = ml_predictor.score(df_test, df_test.MEDV)

In [81]:
model_simple = CatBoostClassifier(
    eval_metric='Precision',
    depth=10,
    iterations=250,
    random_seed=42,
    use_best_model=False
)
model_simple.fit(
    X_train, y_train,
    cat_features=[],
    eval_set=(X_validation, y_validation),
)

0:	learn: 0.7023190	test: 0.6617284	best: 0.6617284 (0)	total: 55.6ms	remaining: 13.8s
1:	learn: 0.7045010	test: 0.6658228	best: 0.6658228 (1)	total: 123ms	remaining: 15.3s
2:	learn: 0.7045010	test: 0.6658228	best: 0.6658228 (2)	total: 219ms	remaining: 18s
3:	learn: 0.7114527	test: 0.6743590	best: 0.6743590 (3)	total: 301ms	remaining: 18.5s
4:	learn: 0.7095079	test: 0.6691729	best: 0.6743590 (3)	total: 358ms	remaining: 17.5s
5:	learn: 0.7168737	test: 0.6729223	best: 0.6743590 (3)	total: 379ms	remaining: 15.4s
6:	learn: 0.7177459	test: 0.6770833	best: 0.6770833 (6)	total: 477ms	remaining: 16.5s
7:	learn: 0.7135851	test: 0.6649616	best: 0.6770833 (6)	total: 585ms	remaining: 17.7s
8:	learn: 0.7105010	test: 0.6666667	best: 0.6770833 (6)	total: 732ms	remaining: 19.6s
9:	learn: 0.7100820	test: 0.6675063	best: 0.6770833 (6)	total: 1.03s	remaining: 24.8s
10:	learn: 0.7111324	test: 0.6675063	best: 0.6770833 (6)	total: 1.13s	remaining: 24.5s
11:	learn: 0.7103118	test: 0.6691729	best: 0.6770833 (

96:	learn: 0.7255179	test: 0.6633166	best: 0.6770833 (6)	total: 19.1s	remaining: 30.1s
97:	learn: 0.7267168	test: 0.6633166	best: 0.6770833 (6)	total: 21.1s	remaining: 32.8s
98:	learn: 0.7261625	test: 0.6633166	best: 0.6770833 (6)	total: 21.3s	remaining: 32.5s
99:	learn: 0.7280000	test: 0.6633166	best: 0.6770833 (6)	total: 22.4s	remaining: 33.7s
100:	learn: 0.7289412	test: 0.6633166	best: 0.6770833 (6)	total: 25.6s	remaining: 37.8s
101:	learn: 0.7290292	test: 0.6616541	best: 0.6770833 (6)	total: 25.7s	remaining: 37.3s
102:	learn: 0.7292844	test: 0.6616541	best: 0.6770833 (6)	total: 29s	remaining: 41.4s
103:	learn: 0.7298824	test: 0.6616541	best: 0.6770833 (6)	total: 32s	remaining: 44.9s
104:	learn: 0.7298824	test: 0.6616541	best: 0.6770833 (6)	total: 32.3s	remaining: 44.6s
105:	learn: 0.7300094	test: 0.6616541	best: 0.6770833 (6)	total: 32.9s	remaining: 44.7s
106:	learn: 0.7306065	test: 0.6616541	best: 0.6770833 (6)	total: 33.2s	remaining: 44.4s
107:	learn: 0.7318909	test: 0.6600000	be

190:	learn: 0.7535698	test: 0.6593137	best: 0.6770833 (6)	total: 1m 23s	remaining: 25.7s
191:	learn: 0.7535698	test: 0.6593137	best: 0.6770833 (6)	total: 1m 23s	remaining: 25.1s
192:	learn: 0.7535632	test: 0.6593137	best: 0.6770833 (6)	total: 1m 23s	remaining: 24.6s
193:	learn: 0.7551772	test: 0.6593137	best: 0.6770833 (6)	total: 1m 26s	remaining: 24.9s
194:	learn: 0.7549425	test: 0.6593137	best: 0.6770833 (6)	total: 1m 26s	remaining: 24.4s
195:	learn: 0.7549425	test: 0.6593137	best: 0.6770833 (6)	total: 1m 26s	remaining: 23.8s
196:	learn: 0.7554023	test: 0.6593137	best: 0.6770833 (6)	total: 1m 26s	remaining: 23.3s
197:	learn: 0.7552898	test: 0.6593137	best: 0.6770833 (6)	total: 1m 26s	remaining: 22.8s
198:	learn: 0.7550551	test: 0.6593137	best: 0.6770833 (6)	total: 1m 26s	remaining: 22.2s
199:	learn: 0.7563218	test: 0.6593137	best: 0.6770833 (6)	total: 1m 26s	remaining: 21.7s
200:	learn: 0.7565578	test: 0.6593137	best: 0.6770833 (6)	total: 1m 26s	remaining: 21.2s
201:	learn: 0.7565578

In [82]:
predicted = model_simple.predict(y)

In [ ]:
predicted = model.predict(y)

In [124]:
predicted = ml_predictor.predict(test)

In [125]:
sub2 = pd.DataFrame({'PERID':test.PERID, 'Criminal':[int(i) for i in predicted]})
sub2 = sub2[['PERID', 'Criminal']]
sub2.to_csv('sub_edit2.csv', index=False)